# Part 1 HGVS Non Compliance Issue  


### Pavlos Antoniou
### 31/07/2021
### pa10@sanger.ac.uk

## Case 1 : Fusion annotation

In [1]:
import re
import csv

#NM_152263.2:r.-115_775::NM_002609.3:r.1580_*1924

class Fusion:

    def __init__(self,transcript1,location1,transcript2,location2):
        self.transcript1=transcript1
        self.location1=location1
        self.transcript2=transcript2
        self.location2=location2
        
    def __str__(self):
        return f"{self.transcript1}:r.{self.location1[0]}_{self.location1[1]}::{self.transcript2}:r.{self.location2[0]}_{self.location2[1]}"


def extract_genomic_coordinates(VEP_annotation_file):
    with open(VEP_annotation_file, 'r') as csvfile:
        reader=csv.reader(csvfile, delimiter="\t")

        headers = next(reader)[:1]
        #print(headers)
        for row in reader:
            if row[0].startswith("#"):
                continue
            variant=row[2]
            
            chromosome=row[0]
            start=row[1]
            ref=row[3]
            alt=row[4]
            stop=len(alt)+int(start)-1
        
            print("Variant\tChromosome\tStart\tStop\tREF\tALT")
            print(f"{variant}\t{chromosome}\t{start}\t{stop}\t{ref}\t{alt}\n")

        

def fusion_to_hgvs(annotation):
    transcript_pattern=re.compile('ENST(\d+)\.(\d+)\(')
    positions_pattern=re.compile('\:r.(\d+)_(\d+)(_*)')
    transcripts_list=[]
    positions_list=[]

    transcripts=transcript_pattern.findall(annotation)
    for transcript in transcripts:
        transcript_name=f"ENST{transcript[0]}.{transcript[1]}"
        transcripts_list.append(transcript_name)
    
    positions=positions_pattern.findall(annotation)
    for pos in positions:
        #print(pos)
        (pos1,pos2)=pos[0],pos[1]
        positions_list.append((pos1,pos2))

    f=Fusion(transcripts_list[0],positions_list[0],transcripts_list[1],positions_list[1])
    print(f)



In [2]:
print("Case 1")
non_hgvs_annotation="ENST00000318522.5(EML4):r.1_1751_ENST00000389048.3(ALK):r.4080_6220"
print(f"Cosmic fusion syntax: {non_hgvs_annotation}:")
print("HGVS annotation: ", end="")
fusion_to_hgvs(non_hgvs_annotation)


Case 1
Cosmic fusion syntax: ENST00000318522.5(EML4):r.1_1751_ENST00000389048.3(ALK):r.4080_6220:
HGVS annotation: ENST00000318522.5:r.1_1751::ENST00000389048.3:r.4080_6220


## Case 2

## Genomic coordinates of mutation

In [3]:
print("Run VEP at https://www.ensembl.org/Multi/Tools/VEP?db=core ")
print("with HGVS accession: ENST00000540549.1:c.355_356insATGG")
print("Genomic coordinates (build 38):")
VEP_annotation="../vep_results/zs5zkjDyQn8IM2ZB.vcf"
extract_genomic_coordinates(VEP_annotation_file=VEP_annotation)

Run VEP at https://www.ensembl.org/Multi/Tools/VEP?db=core 
with HGVS accession: ENST00000540549.1:c.355_356insATGG
Genomic coordinates (build 38):
Variant	Chromosome	Start	Stop	REF	ALT
ENST00000540549.1:c.355_356insATGG	4	105234297	105234301	A	AATGG



### Run VEP command line to retrieve genomic coordinates for cds mutations

We can run VEP using the docker container **ensembl-vep** \
We need to have the mutation in valid HGVS format. 




In [4]:
#INPUT for VEP
with open('../vep_results/input.txt','r') as f:
    for line in f:
        print(line)


ENST00000540549.1:c.355_356insATGG



We can then run VEP with the above input file. \
**input: input.txt \
output: insertion.vcf** \
The command to run VEP for an HGVS mutation: \
**docker run -t -i -v $HOME/vep_data:/opt/vep/.vep ensemblorg/ensembl-vep
./vep --cache  --force_overwrite --vcf  --input_file /opt/vep/.vep/input.txt  --output_file /opt/vep/.vep/insertion.vcf** \


\
We can then extract the genomic coordinates from the VCF file. Based on the type of mutation(SNP,indel) we can calculate the end position of the mutation. (i.e if insertion we can add the length of the inserted sequence to the start position of the mutation) 



In [5]:
print("Run VEP at https://www.ensembl.org/Multi/Tools/VEP?db=core ")
print("with HGVS accession: ENST00000540549.1:c.355_356insATGG")
print("Genomic coordinates (build 38):")
VEP_annotation="../vep_results/insertion.vcf"
extract_genomic_coordinates(VEP_annotation_file=VEP_annotation)

Run VEP at https://www.ensembl.org/Multi/Tools/VEP?db=core 
with HGVS accession: ENST00000540549.1:c.355_356insATGG
Genomic coordinates (build 38):
Variant	Chromosome	Start	Stop	REF	ALT
ENST00000540549.1:c.355_356insATGG	4	105234297	105234301	A	AATGG



## Case 3


In [6]:
print("Cosmic Deletion: ARID1B_ENST00000275248 c.1_6696del6696 ")
print("_____________")
my_text="""This mutation has been found using both DNA microarray and sequencing based analysis of pancreatic cancer genomes according to the study that reported it: 
https://cancer.sanger.ac.uk/cosmic/study/overview?paper_id=45519
Therefore if sequencing was performed the breakpoints are known precisely and I do not need to use (?) 
in the HGVS definition. 
I would therefore use the following HGVS standard for whole gene deletion:
NC_000023.11:g.(31060227_31100351)_(33274278_33417151)del (microarray example with its probe locations)
    """

print(my_text)
print("The HGVS whole gene deletion for this gene:")
print("ENST00000275248.4:g.(1)_(6696)del")

Cosmic Deletion: ARID1B_ENST00000275248 c.1_6696del6696 
_____________
This mutation has been found using both DNA microarray and sequencing based analysis of pancreatic cancer genomes according to the study that reported it: 
https://cancer.sanger.ac.uk/cosmic/study/overview?paper_id=45519
Therefore if sequencing was performed the breakpoints are known precisely and I do not need to use (?) 
in the HGVS definition. 
I would therefore use the following HGVS standard for whole gene deletion:
NC_000023.11:g.(31060227_31100351)_(33274278_33417151)del (microarray example with its probe locations)
    
The HGVS whole gene deletion for this gene:
ENST00000275248.4:g.(1)_(6696)del
